In [10]:
import hashlib

def parallel_hash128(message: bytes, block_size: int = 128) -> bytes:
    """
    Simple demonstration of ParallelHash128 (NIST SP 800-185).
    Uses cSHAKE128 from hashlib (Python 3.6+ with shake support).
    Educational implementation – not fully optimized or certified.
    """
    # Customization string S = "ParallelHash"
    S = b"ParallelHash"
    
    # Encode block size B in bits as 2-byte big-endian
    B_bits = block_size * 8
    encoded_B = B_bits.to_bytes(2, 'big')
    
    if len(message) <= block_size:
        # Direct hash for small messages
        hasher = hashlib.shake_128()
        hasher.update(b"")  # Empty N
        hasher.update(S)
        hasher.update(message + encoded_B)
        return hasher.digest(16)  # 128-bit output
    
    # Split into blocks and hash in parallel (simulated here sequentially)
    n = (len(message) + block_size - 1) // block_size
    blocks = [message[i*block_size:(i+1)*block_size] for i in range(n)]
    
    chain = b""
    for block in blocks:
        block_hasher = hashlib.shake_128()
        block_hasher.update(encoded_B)  # bytepad
        block_hasher.update(block)
        block_hash = block_hasher.digest(16)
        chain += block_hash
    
    # Final cSHAKE on chain
    final_hasher = hashlib.shake_128()
    final_hasher.update(b"")
    final_hasher.update(S)
    final_hasher.update(chain + encoded_B)
    return final_hasher.digest(16)

# Demo
if __name__ == "__main__":
    message = b"In the beginning God created the heavens and the earth"
    hash_value = parallel_hash128(message)
    
    print("Message:", message.decode())
    print("ParallelHash128 (128 bits / 16 bytes, hex):", hash_value.hex())

Message: In the beginning God created the heavens and the earth
ParallelHash128 (128 bits / 16 bytes, hex): c4fd93c6ee582b7b93b31e1dc399c50e
